In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Lasso

In [2]:
# Define parameters for LASSO
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10]
#max_iter = [500, 1000, 10000, 100000]
selection = ['cyclic', 'random']
fit_intercept = [True, False]
tol = [0.0001, 0.001, 0.01, 0.1, 1, 10]
warm_start = [True, False]
#positive = [True, False]
#random_state = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
copy_X = [True, False]

# Create the random grid
random_grid = {'alpha': alpha,
               #'max_iter': max_iter,
               'selection': selection,
               'fit_intercept': fit_intercept,
               'tol': tol,
               'warm_start': warm_start,
               #'positive': positive,
               #'random_state': random_state,
               'copy_X': copy_X}

In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_032022_032023_taxi_rides.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

data['hour_of_day'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_month'] = data['date'].dt.day
data['month_of_year'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# make a weekend column
data['weekend'] = 0
data.loc[data['day_of_week'] == 5, 'weekend'] = 1
data.loc[data['day_of_week'] == 6, 'weekend'] = 1

#make a monday column
data['monday'] = 0
data.loc[data['day_of_week'] == 0, 'monday'] = 1

#make a tuesday column
data['tuesday'] = 0
data.loc[data['day_of_week'] == 1, 'tuesday'] = 1

#make a wednesday column
data['wednesday'] = 0
data.loc[data['day_of_week'] == 2, 'wednesday'] = 1

#make a thursday column
data['thursday'] = 0
data.loc[data['day_of_week'] == 3, 'thursday'] = 1

#make a friday column
data['friday'] = 0
data.loc[data['day_of_week'] == 4, 'friday'] = 1

#make a saturday column
data['saturday'] = 0
data.loc[data['day_of_week'] == 5, 'saturday'] = 1

#make a sunday column
data['sunday'] = 0
data.loc[data['day_of_week'] == 6, 'sunday'] = 1


# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Define the feature columns and the target column
feature_cols = ['time', 'hour_of_day', 'day_of_week', 'day_of_month', 'month_of_year', 'year', 'weekend', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
target_col = 'y'

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})

In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = Lasso()

search = RandomizedSearchCV(estimator = model, 
                               param_distributions = random_grid, 
                               n_iter = 25, 
                               cv = tscv,
                               refit=True, 
                               verbose=3, 
                               random_state=42, 
                               n_jobs = -1, 
                               scoring=scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                               error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.056e+05, tolerance: 7.413e+04
  model = cd_fast.enet_coordinate_descent(


[CV 1/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.117 total time=   0.1s
[CV 3/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.060 total time=   0.1s
[CV 4/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-0.992 total time=   0.0s
[CV 2/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.387 total time=   0.1s
[CV 1/5] END alpha=0.1, copy_X=False, fit_intercept=True, selection=cyclic, tol=10, warm_start=True;, score=-1.131 total time=   0.0s
[CV 5/5] END alpha=0.001, copy_X=True, fit_intercept=True, selection=random, tol=10, warm_start=True;, score=-1.243 total time=   0.0s
[CV 2/5] END alpha=0.1, copy_X=False, fit_intercept=True, selection=cyclic, tol=10, warm_start=True;, score=-1.390 total time=   0.0s
[CV 3/5] END alpha=0.1, copy_X=False, fit_intercept=Tr

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.075e+05, tolerance: 7.413e+03
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.090e+05, tolerance: 1.525e+04
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

[CV 1/5] END alpha=0.01, copy_X=False, fit_intercept=False, selection=cyclic, tol=0.0001, warm_start=False;, score=-1.117 total time=   0.0s
[CV 3/5] END alpha=10, copy_X=True, fit_intercept=False, selection=random, tol=1, warm_start=True;, score=-1.223 total time=   0.0s
[CV 4/5] END alpha=10, copy_X=True, fit_intercept=False, selection=random, tol=1, warm_start=True;, score=-1.662 total time=   0.0s
[CV 2/5] END alpha=0.01, copy_X=False, fit_intercept=False, selection=cyclic, tol=0.0001, warm_start=False;, score=-1.387 total time=   0.0s
[CV 2/5] END alpha=0.001, copy_X=False, fit_intercept=True, selection=cyclic, tol=1, warm_start=False;, score=-1.387 total time=   0.0s
[CV 5/5] END alpha=0.01, copy_X=False, fit_intercept=False, selection=cyclic, tol=0.0001, warm_start=False;, score=-0.937 total time=   0.0s
[CV 5/5] END alpha=10, copy_X=True, fit_intercept=False, selection=random, tol=1, warm_start=True;, score=-1.026 total time=   0.0s
[CV 3/5] END alpha=0.001, copy_X=False, fit_i

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.961e+05, tolerance: 3.807e+04
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.073e+05, tolerance: 2.905e+04
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

[CV 3/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.0001, warm_start=True;, score=-1.060 total time=   0.1s
[CV 4/5] END alpha=0.0001, copy_X=False, fit_intercept=False, selection=random, tol=0.1, warm_start=True;, score=-1.110 total time=   0.2s
[CV 1/5] END alpha=10, copy_X=True, fit_intercept=False, selection=cyclic, tol=10, warm_start=True;, score=-3.150 total time=   0.0s
[CV 3/5] END alpha=0.0001, copy_X=False, fit_intercept=True, selection=cyclic, tol=0.1, warm_start=False;, score=-1.060 total time=   0.1s
[CV 2/5] END alpha=10, copy_X=True, fit_intercept=False, selection=cyclic, tol=10, warm_start=True;, score=-3.091 total time=   0.0s
[CV 5/5] END alpha=0.01, copy_X=True, fit_intercept=False, selection=random, tol=0.0001, warm_start=True;, score=-0.937 total time=   0.1s
[CV 4/5] END alpha=10, copy_X=True, fit_intercept=False, selection=cyclic, tol=10, warm_start=True;, score=-1.898 total time=   0.0s
[CV 3/5] END alpha=10, copy_X=True, fit_inter

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.904e+05, tolerance: 2.977e+02
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.062e+05, tolerance: 2.239e+05
  model = cd_fast.enet_coordinate_descent(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Lasso(), n_iter=25, n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.001, 0.01, 0.1, 1,
                                                  10],
                                        'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'selection': ['cyclic', 'random'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1, 1,
                                                10],
                                        'warm_start': [True, False]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -1.0968424665528325
Best Hyperparameters: {'warm_start': False, 'tol': 0.01, 'selection': 'random', 'fit_intercept': False, 'copy_X': False, 'alpha': 0.01}
Best Model: Lasso(alpha=0.01, copy_X=False, fit_intercept=False, selection='random',
      tol=0.01)
Best Index: 12
CV Results: {'mean_fit_time': array([0.05541873, 0.01158276, 0.00422182, 0.00387402, 0.00503545,
       0.00370827, 0.00422187, 0.07786341, 0.00472565, 0.00417242,
       0.00861807, 0.00453959, 0.06634803, 0.00337915, 0.00334401,
       0.00443997, 0.05026078, 0.09482498, 0.00444617, 0.00566401,
       0.00911083, 0.09403038, 0.00487385, 0.07332759, 0.00674448]), 'std_fit_time': array([0.01445728, 0.00839835, 0.00192769, 0.0012801 , 0.00173559,
       0.00112709, 0.00121982, 0.03726301, 0.00107347, 0.00075897,
       0.00279275, 0.00180314, 0.03313011, 0.00116888, 0.0010102 ,
       0.00232399, 0.0344789 , 0.05175786, 0.00206977, 0.00331676,
       0.00359895, 0.04650086, 0.0026943 , 0.03524236, 0.00585147